# Federal Employment Data Analysis
## Data Exploration, Cleaning & Transformation

**Dataset:** November 2025 Federal Employment Data (~780MB, 2M+ records)

---

### Quick Start

**1. Install Dependencies:**
```bash
pip install pandas numpy matplotlib seaborn jupyter
```

**2. Get the Data:**
- Download the federal employment CSV file
- Update `RAW_DATA_PATH` in the next cell to point to your file location

**3. Run the Notebook:**
- Run all cells in order
- Processed data will be exported to `../data/processed/`
- View the dashboard by opening `../dashboard/index.html` in a browser

---

### Objectives
1. Load and profile the data efficiently
2. Clean and standardize values
3. Create aggregated datasets for analysis
4. Export processed data for the interactive dashboard

In [ ]:
# Core imports
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.float_format', '{:,.2f}'.format)

# Paths
RAW_DATA_PATH = Path('../data/raw/employment_data.csv')  # Update this path to your data file
PROCESSED_DATA_PATH = Path('../data/processed')
PROCESSED_DATA_PATH.mkdir(parents=True, exist_ok=True)

print('Setup complete!')

## 1. Initial Data Loading

The file is ~780MB, so we'll use chunked reading for memory efficiency.

In [ ]:
# First, let's peek at the structure without loading everything
df_sample = pd.read_csv(RAW_DATA_PATH, sep='|', nrows=1000)
print(f"Columns ({len(df_sample.columns)}):")
print(df_sample.columns.tolist())
print(f"\nSample shape: {df_sample.shape}")

In [ ]:
# View first few rows
df_sample.head(10)

In [ ]:
# Check data types and memory usage of sample
df_sample.info()

In [ ]:
# Count total rows without loading full file
def count_rows(filepath):
    """Count rows efficiently without loading entire file"""
    count = 0
    for chunk in pd.read_csv(filepath, sep='|', chunksize=100000, usecols=[0]):
        count += len(chunk)
    return count

print("Counting total rows (this may take a moment)...")
total_rows = count_rows(RAW_DATA_PATH)
print(f"Total rows in dataset: {total_rows:,}")

## 2. Data Profiling

Understanding what's in each column before cleaning.

In [ ]:
# Profile each column - unique values, nulls, data types
def profile_columns(df):
    """Generate a profile of each column"""
    profile = []
    for col in df.columns:
        profile.append({
            'column': col,
            'dtype': str(df[col].dtype),
            'non_null': df[col].notna().sum(),
            'null_count': df[col].isna().sum(),
            'null_pct': f"{df[col].isna().mean()*100:.1f}%",
            'unique': df[col].nunique(),
            'sample_values': df[col].dropna().head(3).tolist()
        })
    return pd.DataFrame(profile)

profile_df = profile_columns(df_sample)
profile_df

In [ ]:
# Check for REDACTED values across columns
def check_redacted(df):
    """Count REDACTED values in each column"""
    redacted_counts = {}
    for col in df.columns:
        if df[col].dtype == 'object':
            redacted_count = (df[col] == 'REDACTED').sum()
            if redacted_count > 0:
                redacted_counts[col] = redacted_count
    return redacted_counts

redacted = check_redacted(df_sample)
print("Columns with REDACTED values:")
for col, count in redacted.items():
    print(f"  {col}: {count} ({count/len(df_sample)*100:.1f}%)")

In [ ]:
# Examine unique values for key categorical columns
categorical_cols = [
    'age_bracket', 'agency', 'appointment_type', 'education_level',
    'pay_plan', 'supervisory_status', 'work_schedule', 'stem_occupation'
]

for col in categorical_cols:
    if col in df_sample.columns:
        print(f"\n=== {col} ===")
        print(df_sample[col].value_counts().head(10))

## 3. Load Full Dataset

Now that we understand the structure, let's load the full dataset with optimized dtypes.

In [ ]:
# Define optimized dtypes to reduce memory usage
dtype_map = {
    'age_bracket': 'category',
    'agency': 'category',
    'agency_code': 'category',
    'agency_subelement': 'category',
    'agency_subelement_code': 'category',
    'annualized_adjusted_basic_pay': 'object',  # Keep as object due to REDACTED
    'appointment_type': 'category',
    'appointment_type_code': 'category',
    'count': 'int32',
    'duty_station_country': 'category',
    'duty_station_country_code': 'category',
    'duty_station_state': 'category',
    'duty_station_state_abbreviation': 'category',
    'duty_station_state_code': 'category',
    'education_level': 'category',
    'education_level_code': 'category',
    'grade': 'object',  # Mixed types possible
    'length_of_service_years': 'float32',
    'occupational_group': 'category',
    'occupational_group_code': 'category',
    'occupational_series': 'category',
    'occupational_series_code': 'category',
    'pay_plan': 'category',
    'pay_plan_code': 'category',
    'snapshot_yyyymm': 'int32',
    'stem_occupation': 'category',
    'stem_occupation_type': 'category',
    'supervisory_status': 'category',
    'supervisory_status_code': 'category',
    'work_schedule': 'category',
    'work_schedule_code': 'category'
}

print("Loading full dataset with optimized dtypes...")
print("This may take a few moments for a 780MB file...")

df = pd.read_csv(RAW_DATA_PATH, sep='|', dtype=dtype_map, low_memory=False)

print(f"\nLoaded {len(df):,} rows and {len(df.columns)} columns")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

## 4. Data Cleaning

In [ ]:
# Create a clean copy
df_clean = df.copy()

# Convert pay to numeric (REDACTED becomes NaN)
df_clean['pay_numeric'] = pd.to_numeric(
    df_clean['annualized_adjusted_basic_pay'], 
    errors='coerce'
)

# Flag redacted records
df_clean['is_redacted'] = df_clean['annualized_adjusted_basic_pay'] == 'REDACTED'

print(f"Records with salary data: {df_clean['pay_numeric'].notna().sum():,}")
print(f"Records with REDACTED salary: {df_clean['is_redacted'].sum():,}")

In [ ]:
# Clean grade column - extract numeric where possible
df_clean['grade_numeric'] = pd.to_numeric(df_clean['grade'], errors='coerce')

print("Grade distribution (where numeric):")
print(df_clean['grade_numeric'].describe())

In [ ]:
# Create tenure categories
def categorize_tenure(years):
    if pd.isna(years):
        return 'Unknown'
    elif years < 1:
        return '< 1 year'
    elif years < 5:
        return '1-5 years'
    elif years < 10:
        return '5-10 years'
    elif years < 20:
        return '10-20 years'
    elif years < 30:
        return '20-30 years'
    else:
        return '30+ years'

df_clean['tenure_category'] = df_clean['length_of_service_years'].apply(categorize_tenure)
df_clean['tenure_category'] = df_clean['tenure_category'].astype('category')

print("Tenure distribution:")
print(df_clean['tenure_category'].value_counts())

In [ ]:
# Create pay bands
def categorize_pay(pay):
    if pd.isna(pay):
        return 'Unknown/Redacted'
    elif pay < 40000:
        return '< $40K'
    elif pay < 60000:
        return '$40K-$60K'
    elif pay < 80000:
        return '$60K-$80K'
    elif pay < 100000:
        return '$80K-$100K'
    elif pay < 150000:
        return '$100K-$150K'
    elif pay < 200000:
        return '$150K-$200K'
    else:
        return '$200K+'

df_clean['pay_band'] = df_clean['pay_numeric'].apply(categorize_pay)
df_clean['pay_band'] = df_clean['pay_band'].astype('category')

print("Pay band distribution:")
print(df_clean['pay_band'].value_counts())

## 5. Exploratory Analysis

In [ ]:
# Agency overview
agency_summary = df_clean.groupby('agency').agg({
    'count': 'sum',
    'pay_numeric': ['mean', 'median', 'min', 'max'],
    'length_of_service_years': 'mean',
    'grade_numeric': 'mean'
}).round(2)

agency_summary.columns = ['employee_count', 'avg_pay', 'median_pay', 'min_pay', 'max_pay', 'avg_tenure', 'avg_grade']
agency_summary = agency_summary.sort_values('employee_count', ascending=False)

print("Top 20 Agencies by Employee Count:")
agency_summary.head(20)

In [ ]:
# Visualize top agencies
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Employee count
top_agencies = agency_summary.head(15)
axes[0].barh(range(len(top_agencies)), top_agencies['employee_count'])
axes[0].set_yticks(range(len(top_agencies)))
axes[0].set_yticklabels(top_agencies.index)
axes[0].set_xlabel('Employee Count')
axes[0].set_title('Top 15 Agencies by Headcount')
axes[0].invert_yaxis()

# Average pay
axes[1].barh(range(len(top_agencies)), top_agencies['avg_pay'])
axes[1].set_yticks(range(len(top_agencies)))
axes[1].set_yticklabels(top_agencies.index)
axes[1].set_xlabel('Average Pay ($)')
axes[1].set_title('Average Pay by Agency')
axes[1].invert_yaxis()

plt.tight_layout()
plt.savefig('../data/processed/agency_overview.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Pay by education level
education_pay = df_clean.groupby('education_level').agg({
    'count': 'sum',
    'pay_numeric': ['mean', 'median']
}).round(2)
education_pay.columns = ['employee_count', 'avg_pay', 'median_pay']
education_pay = education_pay.sort_values('median_pay', ascending=False)

print("Pay by Education Level:")
education_pay

In [ ]:
# Geographic distribution
state_summary = df_clean.groupby('duty_station_state').agg({
    'count': 'sum',
    'pay_numeric': 'mean'
}).round(2)
state_summary.columns = ['employee_count', 'avg_pay']
state_summary = state_summary.sort_values('employee_count', ascending=False)

print("Top 20 States by Federal Employment:")
state_summary.head(20)

In [ ]:
# Appointment type analysis
appointment_summary = df_clean.groupby('appointment_type').agg({
    'count': 'sum',
    'pay_numeric': 'mean',
    'length_of_service_years': 'mean'
}).round(2)
appointment_summary.columns = ['employee_count', 'avg_pay', 'avg_tenure']
appointment_summary = appointment_summary.sort_values('employee_count', ascending=False)

print("Employment by Appointment Type:")
appointment_summary

In [ ]:
# Supervisory status breakdown
supervisor_summary = df_clean.groupby('supervisory_status').agg({
    'count': 'sum',
    'pay_numeric': 'mean',
    'length_of_service_years': 'mean'
}).round(2)
supervisor_summary.columns = ['employee_count', 'avg_pay', 'avg_tenure']

print("Supervisory Status Breakdown:")
supervisor_summary

In [ ]:
# STEM vs Non-STEM comparison
stem_summary = df_clean.groupby('stem_occupation').agg({
    'count': 'sum',
    'pay_numeric': 'mean',
    'length_of_service_years': 'mean',
    'grade_numeric': 'mean'
}).round(2)
stem_summary.columns = ['employee_count', 'avg_pay', 'avg_tenure', 'avg_grade']

print("STEM vs Non-STEM Comparison:")
stem_summary

## 6. Cross-Tabulation Analysis

Compare any columns against each other.

In [ ]:
# Function to create comparison tables
def compare_columns(df, col1, col2, metric='count', agg_col='count'):
    """
    Create a cross-tabulation comparing two columns.
    metric: 'count', 'sum', 'mean'
    agg_col: column to aggregate (for sum/mean)
    """
    if metric == 'count':
        return pd.crosstab(df[col1], df[col2], values=df['count'], aggfunc='sum', margins=True)
    elif metric == 'mean':
        pivot = df.pivot_table(values=agg_col, index=col1, columns=col2, aggfunc='mean')
        return pivot.round(2)
    elif metric == 'sum':
        pivot = df.pivot_table(values=agg_col, index=col1, columns=col2, aggfunc='sum')
        return pivot

# Example: Age bracket vs Education level (employee counts)
print("Employee Count: Age Bracket vs Education Level")
compare_columns(df_clean, 'age_bracket', 'education_level')

In [ ]:
# Average pay: Agency vs Supervisory Status
print("Average Pay: Top Agencies vs Supervisory Status")
top_10_agencies = agency_summary.head(10).index.tolist()
df_top_agencies = df_clean[df_clean['agency'].isin(top_10_agencies)]
compare_columns(df_top_agencies, 'agency', 'supervisory_status', metric='mean', agg_col='pay_numeric')

In [ ]:
# Tenure by Age Bracket and STEM status
print("Average Tenure: Age Bracket vs STEM Status")
compare_columns(df_clean, 'age_bracket', 'stem_occupation', metric='mean', agg_col='length_of_service_years')

## 7. Export Aggregated Data for Dashboard

Create smaller, pre-aggregated datasets that the HTML dashboard can load quickly.

In [ ]:
# Export 1: Agency summary
agency_export = df_clean.groupby(['agency', 'agency_code']).agg({
    'count': 'sum',
    'pay_numeric': ['mean', 'median', 'std'],
    'length_of_service_years': ['mean', 'median'],
    'grade_numeric': 'mean',
    'is_redacted': 'sum'
}).round(2)
agency_export.columns = ['_'.join(col).strip('_') for col in agency_export.columns]
agency_export = agency_export.reset_index()
agency_export.to_csv(PROCESSED_DATA_PATH / 'agency_summary.csv', index=False)
print(f"Exported agency_summary.csv: {len(agency_export)} agencies")

In [ ]:
# Export 2: State summary
state_export = df_clean.groupby(['duty_station_state', 'duty_station_state_abbreviation']).agg({
    'count': 'sum',
    'pay_numeric': ['mean', 'median'],
    'length_of_service_years': 'mean'
}).round(2)
state_export.columns = ['_'.join(col).strip('_') for col in state_export.columns]
state_export = state_export.reset_index()
state_export.to_csv(PROCESSED_DATA_PATH / 'state_summary.csv', index=False)
print(f"Exported state_summary.csv: {len(state_export)} states/territories")

In [ ]:
# Export 3: Occupation summary
occupation_export = df_clean.groupby(['occupational_group', 'occupational_series', 'stem_occupation']).agg({
    'count': 'sum',
    'pay_numeric': ['mean', 'median'],
    'length_of_service_years': 'mean',
    'grade_numeric': 'mean'
}).round(2)
occupation_export.columns = ['_'.join(col).strip('_') for col in occupation_export.columns]
occupation_export = occupation_export.reset_index()
occupation_export.to_csv(PROCESSED_DATA_PATH / 'occupation_summary.csv', index=False)
print(f"Exported occupation_summary.csv: {len(occupation_export)} occupation series")

In [ ]:
# Export 4: Demographics summary (for charts)
demographics_export = df_clean.groupby(['age_bracket', 'education_level', 'tenure_category']).agg({
    'count': 'sum',
    'pay_numeric': 'mean'
}).round(2)
demographics_export.columns = ['employee_count', 'avg_pay']
demographics_export = demographics_export.reset_index()
demographics_export.to_csv(PROCESSED_DATA_PATH / 'demographics_summary.csv', index=False)
print(f"Exported demographics_summary.csv: {len(demographics_export)} demographic combinations")

In [ ]:
# Export 5: Pay distribution data
pay_distribution = df_clean.groupby(['pay_band', 'agency']).agg({
    'count': 'sum'
}).reset_index()
pay_distribution.to_csv(PROCESSED_DATA_PATH / 'pay_distribution.csv', index=False)
print(f"Exported pay_distribution.csv: {len(pay_distribution)} rows")

In [ ]:
# Export 6: Appointment type summary
appointment_export = df_clean.groupby(['appointment_type', 'agency']).agg({
    'count': 'sum',
    'pay_numeric': 'mean',
    'length_of_service_years': 'mean'
}).round(2)
appointment_export.columns = ['employee_count', 'avg_pay', 'avg_tenure']
appointment_export = appointment_export.reset_index()
appointment_export.to_csv(PROCESSED_DATA_PATH / 'appointment_summary.csv', index=False)
print(f"Exported appointment_summary.csv: {len(appointment_export)} rows")

In [ ]:
# Export 7: Overall statistics (for dashboard header)
overall_stats = {
    'total_employees': int(df_clean['count'].sum()),
    'total_agencies': int(df_clean['agency'].nunique()),
    'total_states': int(df_clean['duty_station_state'].nunique()),
    'avg_salary': round(df_clean['pay_numeric'].mean(), 2),
    'median_salary': round(df_clean['pay_numeric'].median(), 2),
    'avg_tenure': round(df_clean['length_of_service_years'].mean(), 2),
    'pct_redacted': round(df_clean['is_redacted'].mean() * 100, 2),
    'snapshot_date': int(df_clean['snapshot_yyyymm'].iloc[0])
}

import json
with open(PROCESSED_DATA_PATH / 'overall_stats.json', 'w') as f:
    json.dump(overall_stats, f, indent=2)
    
print("\nOverall Statistics:")
for k, v in overall_stats.items():
    print(f"  {k}: {v:,}" if isinstance(v, int) else f"  {k}: {v}")

In [ ]:
# List all exported files
print("\n=== Exported Files ===")
for f in sorted(PROCESSED_DATA_PATH.glob('*')):
    size_kb = f.stat().st_size / 1024
    print(f"  {f.name}: {size_kb:.1f} KB")

## 8. Custom Analysis Functions

Reusable functions for Jon to compare any columns.

In [ ]:
def analyze_by_column(df, group_col, sort_by='employee_count', top_n=20):
    """
    Quick analysis of any categorical column.
    Returns employee count, average pay, and tenure.
    """
    result = df.groupby(group_col).agg({
        'count': 'sum',
        'pay_numeric': ['mean', 'median'],
        'length_of_service_years': 'mean',
        'grade_numeric': 'mean'
    }).round(2)
    result.columns = ['employee_count', 'avg_pay', 'median_pay', 'avg_tenure', 'avg_grade']
    result = result.sort_values(sort_by, ascending=False).head(top_n)
    return result

# Example usage:
print("Top 15 Occupational Series by Employee Count:")
analyze_by_column(df_clean, 'occupational_series', top_n=15)

In [ ]:
def compare_two_groups(df, col, group1, group2):
    """
    Compare statistics between two specific values of a column.
    """
    g1 = df[df[col] == group1]
    g2 = df[df[col] == group2]
    
    comparison = pd.DataFrame({
        'Metric': ['Employee Count', 'Avg Pay', 'Median Pay', 'Avg Tenure', 'Avg Grade'],
        group1: [
            g1['count'].sum(),
            g1['pay_numeric'].mean(),
            g1['pay_numeric'].median(),
            g1['length_of_service_years'].mean(),
            g1['grade_numeric'].mean()
        ],
        group2: [
            g2['count'].sum(),
            g2['pay_numeric'].mean(),
            g2['pay_numeric'].median(),
            g2['length_of_service_years'].mean(),
            g2['grade_numeric'].mean()
        ]
    }).round(2)
    
    comparison['Difference'] = comparison[group2] - comparison[group1]
    return comparison

# Example: Compare STEM vs non-STEM
print("STEM vs Non-STEM Occupations Comparison:")
compare_two_groups(df_clean, 'stem_occupation', 'ALL OTHER OCCUPATIONS', 'STEM OCCUPATIONS')

In [ ]:
def filter_and_analyze(df, **filters):
    """
    Filter data by multiple criteria and return summary.
    
    Example:
        filter_and_analyze(df, agency='DEPARTMENT OF TREASURY', education_level='MASTER\'S DEGREE')
    """
    filtered = df.copy()
    for col, value in filters.items():
        if col in filtered.columns:
            filtered = filtered[filtered[col] == value]
    
    if len(filtered) == 0:
        return "No records match the specified filters."
    
    summary = {
        'Records': len(filtered),
        'Employee Count': filtered['count'].sum(),
        'Avg Pay': filtered['pay_numeric'].mean(),
        'Median Pay': filtered['pay_numeric'].median(),
        'Avg Tenure': filtered['length_of_service_years'].mean(),
        'Avg Grade': filtered['grade_numeric'].mean()
    }
    return pd.Series(summary).round(2)

# Example: Treasury employees with Master's degree
print("Treasury Department - Master's Degree Holders:")
filter_and_analyze(df_clean, agency='DEPARTMENT OF TREASURY', education_level="MASTER'S DEGREE")

## Next Steps

The processed data files are now ready in `../data/processed/`:

1. **agency_summary.csv** - Per-agency statistics
2. **state_summary.csv** - Per-state statistics  
3. **occupation_summary.csv** - Per-occupation statistics
4. **demographics_summary.csv** - Age/education/tenure breakdowns
5. **pay_distribution.csv** - Pay bands by agency
6. **appointment_summary.csv** - Appointment types by agency
7. **overall_stats.json** - High-level dashboard metrics

These files will power the interactive HTML dashboard.